In [5]:
import minsearch
import json
import requests
import pandas as pd
import json

In [4]:
# response = requests.get('https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json')

In [3]:
# raw = response.json()

In [6]:
with open('../documents.json') as f:
    raw = json.load(f)

In [8]:
documents = []

In [9]:
for course_dict in raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [10]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [11]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
)

In [12]:
q = "this course has already started, can I still enroll?"

In [13]:
index.fit(documents)

In [14]:
boost = {'question': 3.0, 'section': 0.5}

In [15]:
results = index.search(
    query=q,
    boost_dict=boost,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    num_results=5
)

In [16]:
from openai import OpenAI

In [17]:
client = OpenAI()

In [18]:
res  = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': q}]
)

In [19]:
prompt = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}

""".strip()

In [20]:
res

ChatCompletion(id='chatcmpl-Bg5WFyfsfOH8fVF4MGRFPUq6zzyHf', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Whether you can still enroll in a course that has already started depends on several factors, including the specific policies of the institution or platform offering the course, the type of course, and the point in time at which you're trying to enroll. Here are some steps you can take to find out if you can still join:\n\n1. **Check the Course Details:** Look at the course page or syllabus to see if there is a specific enrollment deadline.\n\n2. **Contact the Instructor or Institution:** Reach out to the course instructor or the institution's admissions office to inquire about late enrollment possibilities.\n\n3. **Review the Platform's Policies:** If it's an online course offered through a platform like Coursera, edX, or another, check the platform's FAQs or support pages for information on enrolling late.\n\n4. **Consider Au

In [21]:
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [22]:
prompt = prompt.format(question=q, context=context).strip()

In [23]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: this course has already started, can I still enroll?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - What are the prerequisites for this course?
answer: GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace 

In [24]:
res  = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': 'can i still enroll?'}]
)
print(res.choices[0].message.content)

I'm not sure which enrollment you're referring to. Could you please provide more details or specify the program, course, or event you're interested in? That way, I can give you more accurate information or guidance.


In [25]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: this course has already started, can I still enroll?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - What are the prerequisites for this course?
answer: GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace 

In [26]:
res  = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': prompt}]
)
print(res.choices[0].message.content)

Yes, you can still enroll in the course after it has started. You are eligible to submit the homeworks even if you haven't registered, but keep in mind that there will be deadlines for turning in the final projects.


In [27]:
def min_search(query):
    boost = {'question': 3.0, 'section': 0.5}
    results = index.search(
        query=query,
        boost_dict=boost,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        num_results=5
    )

    return results

In [28]:
results = min_search('how do I run Kafka?')

In [29]:
def build_prompt(query, search_results):
    prompt = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
    """.strip()
    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt.format(question=query, context=context).strip()
    return prompt

In [30]:
def llm(prompt):
    res  = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', 'content': prompt}]
    )
    return res.choices[0].message.content

In [31]:
def rag(query, search_func=min_search):
    search_results = search_func(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer


In [32]:
rag("Kafka")

'If you are encountering the error "ModuleNotFoundError: No module named \'kafka.vendor.six.moves\'" when trying to run "from kafka import KafkaProducer" in Jupyter Notebook for Module 6 Homework, it is likely a compatibility issue with Python 3.12.1 and kafka-python-2.0.2. To resolve this, you should:\n\n1. Uninstall the existing kafka-python package if it is already installed:\n   ```\n   pip uninstall kafka-python\n   ```\n2. Install the compatible version directly from the GitHub repository:\n   ```\n   pip install git+https://github.com/dpkp/kafka-python.git\n   ```\n\nThis should help resolve the compatibility issue and run your code without the "ModuleNotFoundError".'

In [33]:
rag("When can I expect to receive the confirmation email?")

"You don't need to wait for a confirmation email. You're accepted and can start learning and submitting homework without registering, as registration is just used to gauge interest before the start date."

## Elasticsearch

In [34]:
from elasticsearch import Elasticsearch

ModuleNotFoundError: No module named 'elasticsearch'

In [54]:
es_client = Elasticsearch(
    hosts='http://localhost:9200'
)

In [58]:
index_settings = dict(
    settings=dict(
        number_of_shards=1,
        number_of_replicas=0
    ),
    mappings=dict(
        properties=dict(
            text=dict(type='text'),
            section=dict(type='section'),
            question=dict(type='text'),
            course=dict(type='keyword')
        )
    )
)
index_name = 'course-questions'

In [59]:
es_client.indices.create(index=index_name, body=index_settings)

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x0000019EBF1B4580>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it))

In [60]:
from tqdm.auto import tqdm

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name,document=doc)

In [62]:
query = "How do I run Kafka?"

In [63]:
search_query = dict(
    size=5,
    query=dict(
        bool=dict(
            must=dict(
                multi_match=dict(
                    query=query,
                    fields=['question^3', 'text', 'section'],
                    type='best_fields'
                )
            ),
            filter=dict(
                term=dict(
                    course='data-engineering-zoomcamp'
                )
            )
        )
    ),
)
search_query

{'size': 5,
 'query': {'bool': {'must': {'multi_match': {'query': 'How do I run Kafka?',
     'fields': ['question^3', 'text', 'section'],
     'type': 'best_fields'}},
   'filter': {'term': {'course': 'data-engineering-zoomcamp'}}}}}

In [ ]:
res = es_client.search(index=index_name, body=search_query)

In [ ]:
result_docs = []

In [ ]:
for hit in res['hits']['hits']:
    result_docs.append(hit['_source'])

In [1]:
def elastic_search(query):
    search_query = dict(
        size=5,
        query=dict(
            bool=dict(
                must=dict(
                    multi_match=dict(
                        query=query,
                        fields=['question^3', 'text', 'section'],
                        type='best_fields'
                    )
                ),
                filter=dict(
                    term=dict(
                        course='data-engineering-zoomcamp'
                    )
                )
            )
        ),
    )
    res = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in res['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs